In [2]:
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
from matplotlib.patches import Arc
import seaborn as sns

In [87]:
class GetStatsbombData:
    def __init__(self):
        self.competition_file = '/Users/matthiashugli/Virtualenvs/youth-base/youth-base/goalkeeping/open-data/data/competitions.json'

    def get_competition(self, league_name):
        # league_name = input("Choose League: Champions League, \
        #                         FA Women's Super League, FIFA World Cup, \
        #                             La Liga, NWSL, Premier League, UEFA Euro, Women's World Cup")
        comp_file = pd.read_json(self.competition_file)
        competitions = pd.DataFrame(comp_file)
        self.competition_ids = str(competitions[competitions['competition_name'] == (league_name)]['competition_id'].unique()[0])
        
    def get_match_id(self):
        match_files = '/Users/matthiashugli/Virtualenvs/youth-base/youth-base/goalkeeping/open-data/data/matches/' \
            + self.competition_ids + '/*.json'
        matches = pd.DataFrame()
        for filename in glob.glob(match_files):
            file = pd.read_json(filename)
            df = pd.DataFrame(file)
            matches = pd.concat([matches, df])
            self.match_ids = pd.Series(matches['match_id']).tolist()    

    def get_event_files(self, league_name):
        self.get_competition(league_name=league_name)
        self.get_match_id()
        event_path = '/Users/matthiashugli/Virtualenvs/youth-base/youth-base/goalkeeping/open-data/data/events/'
        events = pd.DataFrame()
        for i in self.match_ids:
            filename = event_path + str([i][0]) + '.json'
            file = pd.read_json(str(filename))
            df = pd.DataFrame(file)
            df['event_name'] = [v['name'] for k, v in df['type'].items()]
            df['event_detail'] = [v['name'] for k, v in df['play_pattern'].items()]
            df.drop(['type', 'play_pattern'], axis=1, inplace=True)
            events = pd.concat([events, df])
        
        return events

            # + matches + '.json'
        
        # df = pd.DataFrame()
        # for filename in glob.glob(event_files):
        #     file = pd.read_json(filename)
        #     file.insert(1, 'filename', filename)
        #     df = df.append(file)
        #     df['event_name'] = [v['name'] for k, v in df['type'].items()]
        #     df['event_detail'] = [v['name'] for k, v in df['play_pattern'].items()]
        #     df.drop(['type', 'play_pattern'], axis=1, inplace=True)
        #     return df

In [140]:
x = GetStatsbombData()
# competitions = x.get_competition('La Liga')
# x.get_match_id()
events = x.get_event_files('UEFA Euro')
events = events[(events['shot'].notna()) | (events['goalkeeper'].notna())]\
    [['id', 'related_events', 'player', 'shot', 'goalkeeper']]

In [184]:
events['player_name'] = [v['name'] for k, v in events['player'].items()]
#events['shot_loc'] = [v['statsbomb_xg'] for k, v in events['shot'].items()]

In [185]:
gk_events = events[(events['goalkeeper'].notna()) & (events['related_events'].notna())][['id', 'related_events', 'player_name', 'goalkeeper']]
gk_events['related_events'] = [x[0] for x in gk_events['related_events']]

,id,related_events,player_name,goalkeeper
50,4e5ffb57-b5c8-44f0-b33f-8d21dfab5a77,91f2f8aa-4ee8-4593-97a6-1a7862a7cca5,Lukáš Hrádecký,"{'end_location': [4.0, 38.1], 'type': {'id': 3..."
431,defbd524-173b-49cd-aaa3-f2c279e2f4db,0bb2daca-ab0b-497c-ac59-3cc559be6fb7,Lukáš Hrádecký,"{'end_location': [2.6, 39.2], 'type': {'id': 3..."
753,dcd88480-2136-4b1f-abff-67a5e351907c,631e53b0-ecfd-4b73-b6c1-51c2d63e4015,Matvey Safonov,"{'outcome': {'id': 48, 'name': 'Clear'}, 'type..."
857,609291e4-de5e-421f-b4b7-33564421beff,1215b0dc-81fb-4416-b9a6-0faf150a4e8d,Matvey Safonov,"{'end_location': [6.0, 37.9], 'position': {'id..."
865,7f642e88-7ab3-4eb3-bcd4-386010e0a6dd,1ce72d1b-a7c5-4eb3-965f-61621b798f35,Matvey Safonov,"{'end_location': [2.0, 42.0], 'type': {'id': 3..."
...,...,...,...,...
2659,6f91b1d9-9c2d-42a1-a083-f13325452919,49e47b32-f7c6-497b-8b36-89d116a3e00d,Danny Ward,"{'end_location': [3.4, 39.0], 'position': {'id..."
2693,af44ba09-bdf6-46ed-84da-88c7af8ff945,ed50056d-0dcf-4c4d-af1b-acbcfa42ec26,Yann Sommer,"{'end_location': [1.0, 40.0], 'type': {'id': 3..."
2838,dc754bd6-5fc6-4710-b1cc-c7cfeeef98d8,1fda187c-4b9e-4dbf-b9e1-6f12f03edb69,Yann Sommer,"{'end_location': [3.0, 41.2], 'position': {'id..."
2867,35efe381-cc11-4967-ba36-bc83a1b717eb,6caec139-d3c9-41b1-a092-739fbfd8d4dc,Danny Ward,"{'type': {'id': 33, 'name': 'Shot Saved'}, 'bo..."


In [186]:
shot_events = events[events['shot'].notna()][['id', 'shot']]
shot_events['xg'] = [v['statsbomb_xg'] for k, v in shot_events['shot'].items()]
shot_events['end_location'] = [v['end_location'] for k, v in shot_events['shot'].items()]

In [197]:
shots = gk_events.merge(shot_events, how='left', left_on='related_events', right_on='id')
shots = shots[['player_name', 'xg', 'end_location', 'goalkeeper']].dropna()
shots = shots[shots['player_name'].str.contains('Unai', regex=True)]
shots['length'] = [len(l) for l in shots['end_location']]
shots['type'] = [v['type'] for k, v in shots['goalkeeper'].items()]
shots['outcome'] = [v['name'] for k, v in shots['type'].items()]
shots = shots[shots['length'] == 3].reset_index().drop(columns=['index', 'length', 'goalkeeper', 'type'])
shots['end_location'] = [l[1:] for l in shots['end_location']]

In [199]:
shots.to_csv('/Users/matthiashugli/Downloads/unai_shot_locations.csv')